In [ ]:
# cdnod_feature_select.ipynb results: related features
amzn = ["M1SL","M2SL","INCOME_STATEMENT_operatingIncome","CASH_FLOW_cashflowFromInvestment","CASH_FLOW_capitalExpenditures","CASH_FLOW_operatingCashflow","CASH_FLOW_changeInOperatingLiabilities","BALANCE_SHEET_cashAndCashEquivalentsAtCarryingValue"]
amgn = ["CASH_FLOW_changeInOperatingAssets","BALANCE_SHEET_longTermInvestments","INCOME_STATEMENT_depreciation",""]
goog = ['INCOME_STATEMENT_incomeTaxExpense','INCOME_STATEMENT_otherNonOperatingIncome','M2SL','CPIAUCSL','CASH_FLOW_paymentsForRepurchaseOfCommonStock','UNRATE','PPIACO','M1SL','CASH_FLOW_cashflowFromFinancing']
abt = ['INCOME_STATEMENT_interestAndDebtExpense','CASH_FLOW_proceedsFromRepaymentsOfShortTermDebt','CASH_FLOW_changeInOperatingAssets','CASH_FLOW_depreciationDepletionAndAmortization','CASH_FLOW_changeInReceivables','BALANCE_SHEET_currentDebt']
cvs = ['BALANCE_SHEET_totalNonCurrentLiabilities','BALANCE_SHEET_inventory','BALANCE_SHEET_currentDebt','UNRATE','INCOME_STATEMENT_investmentIncomeNet','CASH_FLOW_paymentsForOperatingActivities','BALANCE_SHEET_otherCurrentLiabilities','BALANCE_SHEET_shortTermInvestments', 'CASH_FLOW_depreciationDepletionAndAmortization','INCOME_STATEMENT_nonInterestIncome','INCOME_STATEMENT_interestAndDebtExpense','CASH_FLOW_capitalExpenditures','CASH_FLOW_proceedsFromRepaymentsOfShortTermDebt','BALANCE_SHEET_cashAndCashEquivalentsAtCarryingValue','CASH_FLOW_cashflowFromFinancing','BALANCE_SHEET_propertyPlantEquipment','CASH_FLOW_changeInOperatingAssets','BALANCE_SHEET_otherNonCurrentAssets','CASH_FLOW_changeInOperatingLiabilities','CASH_FLOW_changeInInventory','CASH_FLOW_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet' 'BALANCE_SHEET_intangibleAssetsExcludingGoodwill','CASH_FLOW_paymentsForRepurchaseOfCommonStock', 'INCOME_STATEMENT_totalRevenue','CASH_FLOW_cashflowFromInvestment']
t = ['BALANCE_SHEET_treasuryStock','BALANCE_SHEET_propertyPlantEquipment', 'BALANCE_SHEET_capitalLeaseObligations','BALANCE_SHEET_inventory','CASH_FLOW_paymentsForRepurchaseOfCommonStock','INCOME_STATEMENT_operatingIncome', 'INCOME_STATEMENT_totalRevenue']

# rename macro columns mapping
columns={
        "DATE": "observation_date",
        "M2SL": "Money_Supply_M2",
        "M1SL": "Money_Supply_M1",
        "FEDFUNDS": "Interest_Rate",
        "PPIACO": "PPI",
        "RTWEXBGS": "Real_Dollar_Index",
        "CPIAUCSL": "CPI",
        "UNRATE": "Unemployment_Rate",
        "GDP": "GDP"
    }

In [ ]:
# data prep

def create_df(features, ticker):
    """"take in a list of features and creates a dataframe with those features for a specific company"""
    data_path = f'../data/macro_micro/{ticker}_quarterly.csv'
    macro_features = ['Money_Supply_2', 'Money_Supply_1', 'Interest_Rate', 'PPI', 'Real_Dollar_Index', 'CPI', 'Unemployment_Rate']
    total_features = macro_features + features + ['quarterly_return']